In [72]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [73]:
data = pd.read_csv('./data.csv')

In [74]:
data['Mood'] = ''

In [75]:
for i in range(len(data)):
    if data.at[i, 'label'] <= 0.25:
        data.at[i, 'Mood'] = 'Depressed'
    if data.at[i, 'label'] > 0.25 and data.at[i, 'label'] <= 0.5:
        data.at[i, 'Mood'] = 'Sad'
    if data.at[i, 'label'] > 0.5 and data.at[i, 'label'] <= 0.75:
        data.at[i, 'Mood'] = 'Happy'
    if data.at[i, 'label'] > 0.75:
        data.at[i, 'Mood'] = 'Cheerful'

In [76]:
data = data.drop(['label'], axis=1)

In [77]:
data.head()

,seq,Mood
0,"No, no\r\nI ain't ever trapped out the bando\r...",Happy
1,"The drinks go down and smoke goes up, I feel m...",Happy
2,She don't live on planet Earth no more\r\nShe ...,Depressed
3,"Trippin' off that Grigio, mobbin', lights low\...",Happy
4,"I see a midnight panther, so gallant and so br...",Sad


In [78]:
import re
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [82]:
x = []
sentences = list(data["seq"])
for sen in sentences:
    x.append(preprocess_text(sen))

In [83]:
dummie = pd.get_dummies(data['Mood'])

In [84]:
labels = dummie
y = labels.values

In [85]:
tokenizer = Tokenizer(num_words=500)
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
# x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

# X_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [86]:
x = pad_sequences(x, padding='post', maxlen=maxlen)

In [87]:
from sklearn.model_selection import train_test_split

In [88]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [91]:
y_test

array([[0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1]], dtype=uint8)

In [92]:
import tensorflow as tf

In [101]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 100, input_length=maxlen))
model.add(tf.keras.layers.Conv1D(100, kernel_size=3, padding='valid', strides=1))
model.add(tf.keras.layers.GlobalAvgPool1D())
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

In [102]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=tf.keras.metrics.AUC())

In [103]:
model.fit(x_train[:10000], y_train[:10000], epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
313/313 [==============================] - 97s 298ms/step - loss: 0.5518 - auc_1: 0.6014 - val_loss: 0.5392 - val_auc_1: 0.6380
Epoch 2/5
313/313 [==============================] - 87s 279ms/step - loss: 0.5333 - auc_1: 0.6526 - val_loss: 0.5380 - val_auc_1: 0.6433
Epoch 3/5
313/313 [==============================] - 87s 277ms/step - loss: 0.5271 - auc_1: 0.6681 - val_loss: 0.5393 - val_auc_1: 0.6452
Epoch 4/5
313/313 [==============================] - 87s 279ms/step - loss: 0.5221 - auc_1: 0.6810 - val_loss: 0.5402 - val_auc_1: 0.6415
Epoch 5/5
313/313 [==============================] - 94s 300ms/step - loss: 0.5181 - auc_1: 0.6902 - val_loss: 0.5462 - val_auc_1: 0.6394


In [140]:
model.save('model.h5')

In [108]:
import pickle as pkl

In [110]:
with open('tokenizer.pkl', 'wb') as file:
    pkl.dump(tokenizer, file)

In [112]:
token = pkl.load(open('tokenizer.pkl', 'rb'))

In [120]:
model.predict(x_test)

array([[0.09000677, 0.26851985, 0.23666218, 0.43227065],
       [0.17354369, 0.08711618, 0.25107998, 0.41003358],
       [0.0888713 , 0.20211184, 0.22757375, 0.4615887 ],
       ...,
       [0.12088549, 0.08862215, 0.34926027, 0.3914298 ],
       [0.07784218, 0.14190906, 0.30203137, 0.3868513 ],
       [0.1781581 , 0.11155686, 0.31118095, 0.36500174]], dtype=float32)

In [ ]:
directory = './model.h5'

In [141]:
# WHOLE MODEL
tflite_model = tf.keras.models.load_model('./model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
with open('new_model.h5', 'wb') as file:
    file.write(tflite_save)
    file.close()

INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmplhc1dktx\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmplhc1dktx\assets


In [144]:
from tensorflow.keras.models import load_model
model = load_model('new_model.h5')

OSError: Unable to open file (file signature not found)